In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import torch
from crypto_bot.binance.data_utils import download_data
from crypto_bot.binance.data_utils import load_dataframe
from crypto_bot.model.data import BinaryTradeDataset, MulticlassTradeDataset, plot_df, plot_by_idx
from crypto_bot.utils import save_obj, load_obj

DATA_FOLDER = 'C:/Users/diego/crypto_bot/data'
WEIGHTS_PATH = 'C:/Users/diego/crypto_bot/data/models'

# Download data

In [2]:
download_data(DATA_FOLDER, startDate='2020-02-01', endDate='2021-11-01')

# Load data

In [2]:
train_df = load_dataframe(DATA_FOLDER, '2020-02-01', '2021-09-30')
valid_df = load_dataframe(DATA_FOLDER, '2021-10-01', '2021-10-30')

# Prepare data

In [2]:
from crypto_bot.model.transforms import FilterFeatures, RelativeTransform, ManualNormalizer, ToTensorTransform
from sklearn.pipeline import Pipeline
from crypto_bot.model.data import MulticlassTradeDataset, plot_df, plot_by_idx

features = ['Open', 'High', 'Low', 'Close', 'Volume']
transforms = Pipeline([
    ('filter_features', FilterFeatures(features)),
    ('relative_features', RelativeTransform('Close', ['Open', 'High', 'Low', 'Close'])),
    ('normalize_volume', ManualNormalizer('Volume', 52.77450324672268, 66.79777013828863)),
    ('to_tensor', ToTensorTransform()),
])

## Binary/Multiclass dataset

In [6]:
# Binary
train_dataset = BinaryTradeDataset(train_df, transforms, 0.7, stride=1)
valid_dataset = BinaryTradeDataset(valid_df, transforms, 0.7, stride=1)

# # Multiclass
# train_dataset = MulticlassTradeDataset(train_df, transforms, 0.7, -0.3, stride=1)
# valid_dataset = MulticlassTradeDataset(valid_df, transforms, 0.7, -0.3, stride=1)

train_dataset.precompute()
valid_dataset.precompute()

In [9]:
train_ds_name = '{}/data_loaders/train_binary_dataset.ds'.format(DATA_FOLDER)
valid_ds_name = '{}/data_loaders/valid_binary_dataset.ds'.format(DATA_FOLDER)

# Save
save_obj(train_ds_name, train_dataset)
save_obj(valid_ds_name, valid_dataset)

# # Load
# train_dataset = load_obj(train_ds_name)
# valid_dataset = load_obj(valid_ds_name)

In [4]:
save_obj('{}/transforms/basic_transform.tf'.format(DATA_FOLDER), transforms)

In [ ]:
# Temporary and binary files
*~
*.py[cod]
*.so
*.cfg
!.isort.cfg
!setup.cfg
*.orig
*.log
*.pot
__pycache__/*
.cache/*
.*.swp
*/.ipynb_checkpoints/*
.DS_Store

# Project files
.ropeproject
.project
.pydevproject
.settings
.idea
.vscode
tags

# Package files
*.egg
*.eggs/
.installed.cfg
*.egg-info

# Unittest and coverage
htmlcov/*
.coverage
.coverage.*
.tox
junit*.xml
coverage.xml
.pytest_cache/

# Build and docs folder/files
build/*
dist/*
sdist/*
docs/api/*
docs/_rst/*
docs/_build/*
cover/*
data/*
data_loaders/*
MANIFEST

# Per-project virtualenvs
.venv*/
.conda*/